## Swipe Left (No) or Right (Yes)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import sparse
import sys
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity
from category_encoders import OneHotEncoder

In [2]:
# read in full cupid data
cupid = pd.read_pickle('data/clean_cupid.pkl')
cupid.drop(columns = ['status', 'location'], inplace = True)

In [3]:
# read in grouped cupid data
#cupid_religion = pd.read_pickle('data/cupid_religion.pkl')

cupid_df = pd.read_pickle('data/grouped_cupid.pkl')

In [3]:
cupid.head(3)

,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
0,22,m,straight,a little extra,strictly anything,socially,never,"doesn't have kids, but might want them",likes dogs and likes cats,agnosticism and very serious about it,sometimes
1,35,m,straight,average,mostly other,often,sometimes,"doesn't have kids, but might want them",likes dogs and likes cats,agnosticism but not too serious about it,no
2,38,m,straight,thin,anything,socially,never,doesn't have kids,has cats,atheism,no


---

In [157]:
orientation('gay/bi', 'm', religion9, lowest_age9, highest_age9)

# df_encoded = OneHotEncoder(use_cat_names = True).fit_transform(df)
    
# # make cosine_similarity input (input X)
# cosine_input = pd.DataFrame(df_encoded.iloc[-1]).T

# # drop last encoded row (input Y -- data for input X to reference)
# df_encoded.drop(df_encoded.tail(1).index, inplace = True)

# # cosine_similarity(X, y)
# similarity = cosine_similarity(cosine_input, df_encoded)

# # return top 5 matches
# top5 = pd.DataFrame(similarity.tolist()[0], columns = ['similarity'], index = df_encoded.index).sort_values(by = 'similarity', ascending = False).iloc[:5]

# # return top 5 matches in a df with cosine similarities
# results = pd.DataFrame(columns = cupid.columns)

,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes


In [188]:
# function to ohe, create sparse matrices, and return the cosine similarity based on orientation

def offspring_subset2(invalue_df, orientation_df):
    """
    invalue_df: converted DataFrame of user inputs
    orientation_df: DataFrame of all people of that orientation
    """
    
    # split by offspring preference
    if "doesn't have kid(s), but wants kid(s)" in invalue_df['offspring'].unique():
        return orientation_df[(orientation_df['offspring'] == "has kid(s)") | (orientation_df['offspring'] == "has kid(s) and wants more") | \
                                        (orientation_df['offspring'] == "has kid(s), but doesn't want more")]
    elif "doesn't have kids, and doesn't want any" in invalue_df['offspring'].unique():
        return orientation_df[(orientation_df['offspring'] == "doesn't have kids") | (orientation_df['offspring'] == "doesn't want kids") | \
                                        (orientation_df['offspring'] == "doesn't have kids, and doesn't want any")]
    elif "has kid(s), but doesn't want more" in invalue_df['offspring'].unique():
        return orientation_df[(orientation_df['offspring'] == "doesn't have kids") | (orientation_df['offspring'] == "doesn't have kid(s), but wants kid(s)") | \
                                        (orientation_df['offspring'] == "wants kid(s)")]
    elif "has kid(s) and wants more" in invalue_df['offspring'].unique():
        return orientation_df[(orientation_df['offspring'] == "has kid(s)") | (orientation_df['offspring'] == "has kid(s) and wants more") | \
                                        (orientation_df['offspring'] == "wants kid(s)")]
    
    # cosine similarities
    # return invalue_to_similarity(invalue_df, orientation_df)

In [197]:
test = cupid_df[(cupid_df['sex'] == 'm') & ((cupid_df['orientation'] == 'gay') | (cupid_df['orientation'] == 'bisexual')) & \
             (cupid_df['religion'] == 'agnosticism') & (cupid_df['age'] >= 28) & (cupid_df['age'] <= 32)]

In [198]:
test['offspring'].value_counts()

doesn't have kids                          53
doesn't want kids                           7
doesn't have kid(s), but wants kid(s)       5
wants kid(s)                                1
doesn't have kids, and doesn't want any     1
has kid(s), but doesn't want more           0
has kid(s) and wants more                   0
has kid(s)                                  0
Name: offspring, dtype: int64

In [202]:
try: 
    offspring_subset(invalue_df, test)
except ValueError:
    print("Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment.")

Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment.


C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [178]:
invalue_df = pd.DataFrame(invalue9).T.rename(columns = {i:j for i,j in zip(np.arange(11), cupid_df.columns)})
invalue_df

,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
0,25,m,gay,thin,vegan,no,no,"doesn't have kid(s), but wants kid(s)",likes dogs and cats,agnosticism,sometimes


In [218]:
cosine_input

,age_33,age_29,age_31,age_46,age_32,age_27,age_45,age_49,age_38,age_30,...,religion_other,religion_buddhism,religion_judaism,religion_catholicism,religion_christianity,religion_islam,religion_hinduism,smokes_no,smokes_sometimes,smokes_yes
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [217]:
df_encoded

,age_33,age_29,age_31,age_46,age_32,age_27,age_45,age_49,age_38,age_30,...,religion_other,religion_buddhism,religion_judaism,religion_catholicism,religion_christianity,religion_islam,religion_hinduism,smokes_no,smokes_sometimes,smokes_yes
55,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
69,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
81,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
175,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
180,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59820,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
59834,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
59910,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
59930,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0


In [186]:
# concat to encode in same fashion
df = pd.concat([test, invalue_df])

# ohe
df_encoded = OneHotEncoder(use_cat_names = True).fit_transform(df)

# X
cosine_input = pd.DataFrame(df_encoded.iloc[-1]).T

# Y
df_encoded.drop(df_encoded.tail(1).index, inplace = True)

similarity = cosine_similarity(cosine_input, df_encoded)

top5 = pd.DataFrame(similarity.tolist()[0], columns = ['similarity'], index = df_encoded.index).sort_values(by = 'similarity', ascending = False).iloc[:5]

results = pd.DataFrame(columns = cupid.columns)
for i in top5.index:
    results = results.append(pd.DataFrame(cupid.loc[i]).T)
    
results

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
40578,45,m,gay,average,mostly vegetarian,socially,never,"doesn't have kids, but might want them",likes dogs and likes cats,agnosticism,when drinking
29190,27,m,gay,fit,strictly anything,not at all,never,doesn't have kids,has dogs and likes cats,agnosticism,sometimes
20298,27,m,gay,average,mostly anything,not at all,never,"doesn't have kids, but wants them",has dogs and likes cats,agnosticism and laughing about it,trying to quit
52726,27,m,gay,fit,mostly vegetarian,not at all,never,"doesn't have kids, but wants them",likes dogs and likes cats,agnosticism,no
39446,22,m,gay,thin,strictly vegan,socially,never,doesn't have kids,likes dogs and likes cats,catholicism but not too serious about it,when drinking


In [216]:
similarity.tolist()[0]

[0.36363636363636365,
 0.4545454545454546,
 0.36363636363636365,
 0.2727272727272727,
 0.36363636363636365,
 0.4545454545454546,
 0.2727272727272727,
 0.36363636363636365,
 0.36363636363636365,
 0.36363636363636365,
 0.2727272727272727,
 0.4545454545454546,
 0.2727272727272727,
 0.18181818181818182,
 0.36363636363636365,
 0.2727272727272727,
 0.2727272727272727,
 0.36363636363636365,
 0.36363636363636365,
 0.36363636363636365,
 0.4545454545454546,
 0.36363636363636365,
 0.2727272727272727,
 0.36363636363636365,
 0.2727272727272727,
 0.2727272727272727,
 0.2727272727272727,
 0.5454545454545455,
 0.2727272727272727,
 0.4545454545454546,
 0.2727272727272727,
 0.36363636363636365,
 0.5454545454545455,
 0.4545454545454546,
 0.2727272727272727,
 0.4545454545454546,
 0.4545454545454546,
 0.36363636363636365,
 0.2727272727272727,
 0.2727272727272727,
 0.2727272727272727,
 0.36363636363636365,
 0.4545454545454546,
 0.36363636363636365,
 0.4545454545454546,
 0.36363636363636365,
 0.3636363636363

In [208]:
top5

,similarity
40578,0.636364
29190,0.636364
20298,0.636364
52726,0.636364
39446,0.636364


---

In [112]:
# separates data by orientation

def orientation(orientation, sex, religion, lowest_age, highest_age):
    """
    invalue (list): survey/streamlit app responses
    religion: religion of user
    lowest_age / highest_age: age range preference of partner recommendation
    """
    
    # straight female looking for straight/bi male
    if orientation == 'straight/bi' and sex == 'm':
        return cupid_df[(((cupid_df['sex'] == 'm') & (cupid_df['orientation'] == 'straight')) | ((cupid_df['sex'] == 'm') & (cupid_df['orientation'] == 'bisexual'))) \
                    & (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
    
    # straight male looking for straight/bi female
    elif orientation == 'straight/bi' and sex == 'f':
        return cupid_df[(((cupid_df['sex'] == 'f') & (cupid_df['orientation'] == 'straight')) | ((cupid_df['sex'] == 'f') & (cupid_df['orientation'] == 'bisexual'))) \
                & (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
    
    # gay male looking for gay/bi male
    elif orientation == 'gay/bi' and sex == 'm':
        return cupid_df[(cupid_df['sex'] == 'm') & ((cupid_df['orientation'] == 'gay') | (cupid_df['orientation'] == 'bisexual')) & \
             (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
    
    # gay female looking for gay/bi female
    elif orientation == 'gay/bi' and sex == 'f':
        return cupid_df[(cupid_df['sex'] == 'f') & ((cupid_df['orientation'] == 'gay') | (cupid_df['orientation'] == 'bisexual')) & \
             (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
    
    # bi female looking for bi individual or straight male or gay female
    elif orientation == 'bisexual' and sex == 'f':
        return cupid_df[((cupid_df['sex'] == 'f') & (cupid_df['orientation'] == 'gay') & (cupid_df['religion'] == religion) & \
                         (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)) | ((cupid_df['sex'] == 'm') & (cupid_df['orientation'] == 'straight') & \
                        (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)) | \
                     (cupid_df['orientation'] == 'bisexual')  & (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
    
    # bi male looking for bi individual or straight female or gay male
    elif orientation == 'bisexual' and sex == 'm':
        return cupid_df[((cupid_df['sex'] == 'm') & (cupid_df['orientation'] == 'gay') & (cupid_df['religion'] == religion) & \
                         (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)) | ((cupid_df['sex'] == 'f') & (cupid_df['orientation'] == 'straight') & \
                        (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)) | \
                     (cupid_df['orientation'] == 'bisexual')  & (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]

In [203]:
# performs recommendation

def lover_recommender_test8(invalue, religion, lowest_age, highest_age):
    """
    invalue (list): survey/streamlit app responses
    religion: religion of user
    lowest_age / highest_age: age range preference of partner recommendation
    """
   
    # convert input from array to DataFrame
    invalue_df = pd.DataFrame(invalue).T.rename(columns = {i:j for i,j in zip(np.arange(11), cupid_df.columns)})

    # ----------------
    
    # straight female looking for straight/bi male
    if invalue_df['orientation'].unique()[0] == 'straight' and invalue_df['sex'].unique()[0] == 'f':
        
        # split data by orientation
        orientation_df = orientation('straight/bi', 'm', religion, lowest_age, highest_age)
        
        # call 'invalue_to_similarity' function to return top 5 similarities
        try: 
            return offspring_subset(invalue_df, orientation_df)
        except ValueError:
            return "Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment."
        
        # return top 5 matches + option to see additional matches
        # return more_matches(matches, religion, lowest_age, highest_age)
    
    # straight male looking for straight/bi female
    elif invalue_df['orientation'].unique()[0] == 'straight' and invalue_df['sex'].unique()[0] == 'm':
        
        # split data by orientation
        orientation_df = orientation('straight/bi', 'f', religion, lowest_age, highest_age)

        # call 'invalue_to_similarity' function to return similarities
        try: 
            return offspring_subset(invalue_df, orientation_df)
        except ValueError:
            return "Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment."
        
        # see matches + option to see additional matches
        # return more_matches(matches, religion, lowest_age, highest_age)
    
    # gay male looking for gay/bi male
    elif invalue_df['orientation'].unique()[0] == 'gay' and invalue_df['sex'].unique()[0] == 'm':
        
        # split data by orientation
        orientation_df = orientation('gay/bi', 'm', religion, lowest_age, highest_age)
        
        # call 'invalue_to_similarity' function to return similarities
        try: 
            return offspring_subset(invalue_df, orientation_df)
        except ValueError:
            return "Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment."
        
        # return top 5 matches + option to see additional matches
        # return more_matches(matches, religion, lowest_age, highest_age)
    
    # gay female looking for gay/bi female
    elif invalue_df['orientation'].unique()[0] == 'gay' and invalue_df['sex'].unique()[0] == 'f':
        
        # split data by orientation
        orientation_df = orientation('gay/bi', 'f', religion, lowest_age, highest_age)
        
        # call 'invalue_to_similarity' function to return similarities
        try: 
            return offspring_subset(invalue_df, orientation_df)
        except ValueError:
            return "Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment."
        
        #  return top 5 matches + option to see additional matches
        # return more_matches(matches, religion, lowest_age, highest_age)
    
    # bi female looking for bi individual or gay female or straight male
    elif (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'f'):
        
        # split data by orientation
        orientation_df = orientation('bisexual', 'f', religion, lowest_age, highest_age)
        
        # call 'invalue_to_similarity' function to return similarities
        try: 
            return offspring_subset(invalue_df, orientation_df)
        except ValueError:
            return "Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment."
        
        #  return top 5 matches + option to see additional matches
        # return more_matches(matches, religion, lowest_age, highest_age)    
    
    # bi male looking for bi individual or gay male or straight female
    elif (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'm'):
        
    # split data by orientation
        orientation_df = orientation('bisexual', 'm', religion, lowest_age, highest_age)

        # call 'invalue_to_similarity' function to return similarities
        try: 
            return offspring_subset(invalue_df, orientation_df)
        except ValueError:
            return "Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment."
        
        #  return top 5 matches + option to see additional matches
        # return more_matches(matches, religion, lowest_age, highest_age)

In [204]:
# function to ohe, create sparse matrices, and return the cosine similarity based on orientation

def offspring_subset(invalue_df, orientation_df):
    """
    invalue_df: converted DataFrame of user inputs
    orientation_df: DataFrame of all people of that orientation
    """
    
    # split by offspring preference
    if "doesn't have kid(s), but wants kid(s)" in invalue_df['offspring'].unique():
        orientation_df = orientation_df[(orientation_df['offspring'] == "has kid(s)") | (orientation_df['offspring'] == "has kid(s) and wants more") | \
                                        (orientation_df['offspring'] == "has kid(s), but doesn't want more")]
    elif "doesn't have kids, and doesn't want any" in invalue_df['offspring'].unique():
        orientation_df = orientation_df[(orientation_df['offspring'] == "doesn't have kids") | (orientation_df['offspring'] == "doesn't want kids") | \
                                        (orientation_df['offspring'] == "doesn't have kids, and doesn't want any")]
    elif "has kid(s), but doesn't want more" in invalue_df['offspring'].unique():
        orientation_df = orientation_df[(orientation_df['offspring'] == "doesn't have kids") | (orientation_df['offspring'] == "doesn't have kid(s), but wants kid(s)") | \
                                        (orientation_df['offspring'] == "wants kid(s)")]
    elif "has kid(s) and wants more" in invalue_df['offspring'].unique():
        orientation_df = orientation_df[(orientation_df['offspring'] == "has kid(s)") | (orientation_df['offspring'] == "has kid(s) and wants more") | \
                                        (orientation_df['offspring'] == "wants kid(s)")]
    
    # cosine similarities
    return invalue_to_similarity(invalue_df, orientation_df)

In [64]:
# function to ohe, create sparse matrices, and return the cosine similarity based on orientation

def invalue_to_similarity(invalue_df, orientation_df):
    """
    invalue_df: converted DataFrame of user inputs
    orientation_df: DataFrame of all people of that orientation
    """
    
    # concat input values to orientation df to prep for cosine similarity
    df = pd.concat([orientation_df, invalue_df])

    # ohe
    df_encoded = OneHotEncoder(use_cat_names = True).fit_transform(df)
    
    # make cosine_similarity input (input X)
    cosine_input = pd.DataFrame(df_encoded.iloc[-1]).T
    
    # drop last encoded row (input Y -- data for input X to reference)
    df_encoded.drop(df_encoded.tail(1).index, inplace = True)
    
    # cosine_similarity(X, y)
    similarity = cosine_similarity(cosine_input, df_encoded)
    
    # return top 5 matches
    top5 = pd.DataFrame(similarity.tolist()[0], columns = ['similarity'], index = df_encoded.index).sort_values(by = 'similarity', ascending = False).iloc[:5]
    
    # return top 5 matches in a df with cosine similarities
    results = pd.DataFrame(columns = cupid.columns)

    for i in top5.index:
        results = results.append(pd.DataFrame(cupid.loc[i]).T)
    
    return results

In [96]:
# function for the user to tell which matches they like and to return additional matches, if the user chooses

def more_matches(matches, religion, lowest_age, highest_age):
    invalue_df = pd.DataFrame(columns = cupid.columns)
    
    Flag = 'yes'
    while Flag == 'yes':
        for i in range(len(matches)):
            person = matches.iloc[i]
            if pd.DataFrame(person).T.index[0] not in invalue_df.index:
                print(person)
                answer = str(input('Do you like them? (yes/no)')).lower()
                if answer == 'yes':
                    invalue_df = invalue_df.append(person)
                else:
                    continue
        
        # straight female
        if invalue_df['sex'].unique()[0] == 'f' and invalue_df['orientation'].unique()[0] == 'straight':
            
            # split data by orientation
            orientation_df = orientation('straight/bi', 'f', religion, lowest_age, highest_age)

            matches = invalue_to_similarity(invalue_df, orientation_df)

        # straight male
        elif invalue_df['sex'].unique()[0] == 'm' and invalue_df['orientation'].unique()[0] == 'straight':
            
            # split data by orientation
            orientation_df = orientation('straight/bi', 'm', religion, lowest_age, highest_age)

            matches = invalue_to_similarity(invalue_df, orientation_df)

        # gay female
        elif invalue_df['sex'].unique()[0] == 'f' and invalue_df['orientation'].unique()[0] == 'gay':
            
            # split data by orientation
            orientation_df = orientation('gay', 'f', religion, lowest_age, highest_age)

            matches = invalue_to_similarity(invalue_df, orientation_df)

        # gay male
        elif invalue_df['sex'].unique()[0] == 'm' and invalue_df['orientation'].unique()[0] == 'gay':
            
            # split data by orientation
            orientation_df = orientation('gay', 'm', religion, lowest_age, highest_age)

            matches = invalue_to_similarity(invalue_df, orientation_df)

        # bi female looking for bi individual or gay female or straight male
        elif (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'f'):

            # split data by orientation
            orientation_df = orientation('bisexual', 'f', religion, lowest_age, highest_age)

            matches = invalue_to_similarity(invalue_df, orientation_df)
            
        # bi male looking for bi individual or gay male or straight female
        elif (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'f'):

            # split data by orientation
            orientation_df = orientation('bisexual', 'm', religion, lowest_age, highest_age)

            matches = invalue_to_similarity(invalue_df, orientation_df)

        Flag = str(input('Do you want to see more matches? (yes/no)')).lower()
    
    print("Here's everyone you matched with:")
    return (invalue_df)

In [150]:
cupid_df['offspring'].value_counts()

doesn't have kids                          41250
doesn't have kid(s), but wants kid(s)       7303
has kid(s)                                  3627
doesn't want kids                           2686
doesn't have kids, and doesn't want any     1080
has kid(s), but doesn't want more            702
has kid(s) and wants more                    425
wants kid(s)                                 400
Name: offspring, dtype: int64

In [142]:
# function for the user to tell which matches they like and to return additional matches, if the user chooses

def more_matches2(invalue, profile, religion, lowest_age, highest_age):
    invalue_df = pd.DataFrame(invalue).T.rename(columns = {i:j for i,j in zip(np.arange(11), cupid_df.columns)})
    likes_df = pd.DataFrame(columns = cupid.columns)
#     profile = []
    
#     Flag = 'yes'
#     while Flag:
#         yield matches
        
#         more = str(input('Any of these profile catch your eye?'))
#         while more == 'yes':
#                 user = int(input('Which profile'))
#                 profile.append(user)
#                 more = str(input('Any more profiles?'))
        
    # collect new profiles
    for person in profile:
        likes_df = likes_df.append(cupid_df.loc[person])

    # straight female
    if invalue_df['sex'].unique()[0] == 'f' and invalue_df['orientation'].unique()[0] == 'straight':

        # split data by orientation
        orientation_df = orientation('straight/bi', 'f', religion, lowest_age, highest_age)

        print("Here are more matches:")
        return offspring_subset(likes_df, orientation_df)

    # straight male
    elif invalue_df['sex'].unique()[0] == 'm' and invalue_df['orientation'].unique()[0] == 'straight':

        # split data by orientation
        orientation_df = orientation('straight/bi', 'm', religion, lowest_age, highest_age)

        print("Here are more matches:")
        return offspring_subset(likes_df, orientation_df)

    # gay female
    elif invalue_df['sex'].unique()[0] == 'f' and invalue_df['orientation'].unique()[0] == 'gay':

        # split data by orientation
        orientation_df = orientation('gay', 'f', religion, lowest_age, highest_age)

        print("Here are more matches:")
        return offspring_subset(likes_df, orientation_df)

    # gay male
    elif invalue_df['sex'].unique()[0] == 'm' and invalue_df['orientation'].unique()[0] == 'gay':

        # split data by orientation
        orientation_df = orientation('gay', 'm', religion, lowest_age, highest_age)

        print("Here are more matches:")
        return offspring_subset(likes_df, orientation_df)

    # bi female looking for bi individual or gay female or straight male
    elif (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'f'):

        # split data by orientation
        orientation_df = orientation('bisexual', 'f', religion, lowest_age, highest_age)

        print("Here are more matches:")
        return offspring_subset(likes_df, orientation_df)

    # bi male looking for bi individual or gay male or straight female
    elif (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'f'):

        # split data by orientation
        orientation_df = orientation('bisexual', 'm', religion, lowest_age, highest_age)
        
        print("Here are more matches:")
        return offspring_subset(likes_df, orientation_df)

        
        #Flag = str(input('Do you want to see more matches? (yes/no)')).lower()
            
    
#     print("Here's everyone you matched with:")
#     return (invalue_df)

In [74]:
# input / up + down arrow

# age + age range
age = int(input('How old are you?'))
print("What's your age range?")
lowest_age = int(input('Lowest age?'))
highest_age = int(input('Highest age?'))

# dropdowns
sex = str(input('What gender do you identify as?'))
orientation = str(input('What sexual orientation do you identify with?'))
body_type = str(input("What's your body type?"))
diet = str(input('What does your diet consist of?'))
drinks = str(input('Do you consume alcoholic beverages?'))
drugs = str(input('Do you use drugs?'))

# offspring
# do you have any children? yes/no
any_children = str(input('Do you have any children?'))

# do you want any [more] children? yes/no
more_children = str(input('Do you want any [more] children?'))

if any_children == 'no' and more_children == 'yes':
    offspring = "doesn't have kid(s), but wants kid(s)"
elif any_children == 'no' and more_children == 'no':
    offspring = "doesn't have kids, and doesn't want any"
elif any_children == 'yes' and more_children == 'no':
    offspring = "has kid(s), but doesn't want more"
elif any_children == 'yes' and more_children == 'yes':
    offspring = "has kid(s) and wants more"

pets = str(input("What's your sentiment on dogs and/or cats"))
smokes = str(input("Do you smoke?"))

# religion
religion_matter = str(input('Does religion matter?'))
if religion_matter == 'yes':
    religion = str(input("What's your religion?"))
else:
    religion = "doesn't matter"

invalue = np.array([age, sex, orientation, body_type, diet, drinks, drugs, offspring, pets, religion, smokes])

How old are you? 28


What's your age range?


Lowest age? 24
Highest age? 43
What gender do you identify as? m
What sexual orientation do you identify with? straight
What's your body type? fit
What does your diet consist of? anythin
Do you consume alcoholic beverages? yes
Do you use drugs? no
Do you have any children? no
Do you want any [more] children? no
What's your sentiment on dogs and/or cats likes cats
Do you smoke? no
Does religion matter? yes
What's your religion? christianity


In [100]:
# straight male

# invalue = ['28', 'm', 'straight', 'fit', 'anything', 'yes', 'no', "doesn't have kids, and doesn't want any", 'likes cats', 'christianity', 'no']
# highest_age = 24
# lowest_age = 43
# religion = 'christianity'

lover_recommender_test8(invalue, religion, lowest_age, highest_age)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
1857,26,f,straight,athletic,anything,often,never,doesn't have kids,has cats,christianity,no
26865,28,f,straight,fit,mostly anything,often,never,doesn't have kids,has dogs and likes cats,christianity,no
58045,24,f,straight,athletic,mostly anything,often,never,doesn't have kids,has dogs,christianity and somewhat serious about it,no
56723,26,f,straight,fit,anything,socially,never,doesn't have kids,has cats,christianity,no
25124,27,f,straight,average,mostly anything,rarely,never,"doesn't have kids, and doesn't want any",has cats,christianity,no


In [90]:
# if the user chooses to see more matches -- will 5 profiles one at a time and the user will indicate whether they like the profile ('yes') or not ('no')

more_matches(lover_recommender_test8(invalue, religion, lowest_age, highest_age), religion, lowest_age, highest_age)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


age                           26
sex                            f
orientation             straight
body_type               athletic
diet                    anything
drinks                     often
drugs                      never
offspring      doesn't have kids
pets                    has cats
religion            christianity
smokes                        no
Name: 1857, dtype: object


Do you like them? yes


age                                 28
sex                                  f
orientation                   straight
body_type                          fit
diet                   mostly anything
drinks                           often
drugs                            never
offspring            doesn't have kids
pets           has dogs and likes cats
religion                  christianity
smokes                              no
Name: 26865, dtype: object


Do you like them? yes


...

Do you like them? yes


C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Do you want to see more matches? no


Here's everyone you matched with:


,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
1857,26,f,straight,athletic,anything,often,never,doesn't have kids,has cats,christianity,no
26865,28,f,straight,fit,mostly anything,often,never,doesn't have kids,has dogs and likes cats,christianity,no
58045,24,f,straight,athletic,mostly anything,often,never,doesn't have kids,has dogs,christianity and somewhat serious about it,no
23537,24,f,straight,thin,anything,rarely,never,doesn't have kids,dislikes dogs and dislikes cats,christianity,no
34356,24,f,straight,average,mostly vegetarian,socially,sometimes,doesn't have kids,likes dogs and has cats,christianity,no
9574,24,f,straight,a little extra,mostly anything,socially,sometimes,doesn't have kids,has dogs and likes cats,christianity and somewhat serious about it,no
54512,24,f,straight,average,anything,socially,never,doesn't have kids,likes dogs and likes cats,christianity and somewhat serious about it,no
47445,24,f,straight,average,mostly anything,socially,never,doesn't have kids,likes dogs and likes cats,christianity and very serious about it,no


In [103]:
# straight female

invalue3 = ['37', 'f', 'straight', 'full figured', 'anything', 'yes', 'no', "doesn't have kid(s), but wants kid(s)", 'likes dogs and cats', "doesn't matter", 'no']
highest_age3 = 40
lowest_age3 = 32
religion3 = "doesn't matter"

lover_recommender_test8(invalue3, religion3, lowest_age3, highest_age3)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
7206,34,m,straight,fit,strictly anything,often,never,has a kid,likes dogs and likes cats,catholicism and laughing about it,no
3497,35,m,straight,curvy,mostly anything,not at all,never,"has a kid, and might want more",likes dogs and likes cats,atheism,no
15047,39,m,straight,fit,mostly anything,often,never,"has a kid, and wants more",likes dogs and likes cats,atheism and somewhat serious about it,no
54828,39,m,straight,average,anything,very often,never,"has kids, but doesn't want more",likes dogs and likes cats,atheism,no
56272,38,m,straight,fit,mostly anything,often,never,has kids,likes dogs and has cats,atheism,no


In [113]:
# bisexual male

invalue2 = ['43', 'm', 'bisexual', 'thin', 'anything', 'yes', 'no', "doesn't have kids, and doesn't want any", 'likes dogs', "doesn't matter", 'yes']
highest_age2 = 45
lowest_age2 = 38
religion2 = "doesn't matter"

lover_recommender_test8(invalue2, religion2, lowest_age2, highest_age2)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
3384,38,f,bisexual,fit,strictly anything,often,never,"doesn't have kids, and doesn't want any",likes dogs and has cats,atheism,yes
22877,38,m,bisexual,average,anything,often,never,doesn't have kids,has dogs,atheism,when drinking
25599,39,m,bisexual,athletic,mostly anything,socially,never,doesn't have kids,likes dogs,agnosticism but not too serious about it,no
16740,38,m,bisexual,fit,mostly anything,socially,never,"doesn't have kids, and doesn't want any",likes dogs and likes cats,atheism and somewhat serious about it,no
57203,38,f,bisexual,thin,anything,rarely,never,doesn't have kids,has dogs,atheism,no


In [121]:
# bisexual female

invalue5 = ['26', 'f', 'bisexual', 'full figured', 'anything', 'no', 'no', "has kid(s) and wants more", 'likes dogs and cats', "buddhism", 'yes']
highest_age5 = 30
lowest_age5 = 22
religion5 = "buddhism"

lover_recommender_test8(invalue5, religion5, lowest_age5, highest_age5)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
27413,29,m,straight,athletic,strictly other,socially,never,wants kids,likes dogs and likes cats,buddhism and somewhat serious about it,no
20508,25,m,straight,athletic,mostly other,socially,never,has a kid,dislikes dogs and dislikes cats,buddhism,sometimes


In [122]:
# let's see more matches, since there are only 2..

more_matches(lover_recommender_test8(invalue5, religion5, lowest_age5, highest_age5), religion5, lowest_age5, highest_age5)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


age                                                29
sex                                                 m
orientation                                  straight
body_type                                    athletic
diet                                   strictly other
drinks                                       socially
drugs                                           never
offspring                                  wants kids
pets                        likes dogs and likes cats
religion       buddhism and somewhat serious about it
smokes                                             no
Name: 27413, dtype: object


Do you like them? (yes/no) yes


age                                         25
sex                                          m
orientation                           straight
body_type                             athletic
diet                              mostly other
drinks                                socially
drugs                                    never
offspring                            has a kid
pets           dislikes dogs and dislikes cats
religion                              buddhism
smokes                               sometimes
Name: 20508, dtype: object


Do you like them? (yes/no) yes


...

Do you like them? (yes/no) yes


C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Do you want to see more matches? (yes/no) no


Here's everyone you matched with:


,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
27413,29,m,straight,athletic,strictly other,socially,never,wants kids,likes dogs and likes cats,buddhism and somewhat serious about it,no
20508,25,m,straight,athletic,mostly other,socially,never,has a kid,dislikes dogs and dislikes cats,buddhism,sometimes
51273,25,m,straight,average,anything,socially,never,doesn't have kids,likes cats,buddhism,sometimes
18080,25,m,straight,athletic,anything,socially,never,doesn't have kids,dislikes dogs and dislikes cats,buddhism,no


In [123]:
# bisexual female

invalue6 = ['26', 'f', 'bisexual', 'full figured', 'anything', 'no', 'no', "doesn't have kids, and doesn't want any", 'likes dogs and cats', "buddhism", 'yes']
highest_age6 = 30
lowest_age6 = 22
religion6 = "buddhism"

lover_recommender_test8(invalue6, religion6, lowest_age6, highest_age6)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
31845,27,f,bisexual,curvy,mostly vegetarian,not at all,never,doesn't want kids,likes dogs and likes cats,buddhism and somewhat serious about it,no
50586,28,f,gay,curvy,anything,socially,never,doesn't have kids,likes dogs and likes cats,buddhism and somewhat serious about it,no
30434,30,f,gay,rather not say,anything,not at all,never,doesn't have kids,likes dogs and has cats,buddhism,no
10268,24,f,bisexual,average,anything,socially,sometimes,doesn't have kids,likes dogs and has cats,buddhism and somewhat serious about it,sometimes
42179,30,m,straight,average,mostly anything,not at all,never,doesn't have kids,likes dogs and likes cats,buddhism,no


In [105]:
# gay male

invalue4 = ['30', 'm', 'gay', 'thin', 'vegan', 'yes', 'yes', "doesn't have kids, and doesn't want any", 'likes dogs', "doesn't matter", 'yes']
highest_age4 = 35
lowest_age4 = 27
religion4 = "doesn't matter"

lover_recommender_test8(invalue4, religion4, lowest_age4, highest_age4)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
3344,29,m,gay,thin,strictly anything,desperately,never,doesn't have kids,has dogs and dislikes cats,agnosticism and laughing about it,yes
41087,27,m,gay,thin,strictly vegan,often,sometimes,doesn't want kids,likes dogs and likes cats,atheism and somewhat serious about it,yes
30001,28,m,gay,average,mostly anything,often,never,doesn't have kids,dislikes cats,catholicism but not too serious about it,yes
858,32,m,gay,thin,anything,often,never,doesn't have kids,likes dogs and has cats,other but not too serious about it,yes
4635,35,m,gay,used up,strictly vegetarian,very often,often,doesn't have kids,dislikes dogs and dislikes cats,atheism,yes


In [152]:
# gay female

invalue7 = ['30', 'f', 'gay', 'rather not say', 'vegetarian', 'yes', 'yes', "doesn't have kid(s), but wants kid(s)", 'dislikes dogs and cats', "christianity", 'yes']
highest_age7 = 45
lowest_age7 = 32
religion7 = "christianity"

lover_recommender_test8(invalue7, religion7, lowest_age7, highest_age7)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
25664,35,f,bisexual,curvy,mostly anything,often,never,"has kids, and might want more",has dogs,christianity,yes
49211,32,f,gay,average,anything,socially,never,has a kid,dislikes dogs and dislikes cats,christianity,no
24990,32,f,bisexual,average,anything,rarely,never,has kids,dislikes dogs and dislikes cats,christianity and somewhat serious about it,sometimes
42459,43,f,bisexual,athletic,mostly anything,socially,never,"has kids, but doesn't want more",likes dogs and has cats,christianity and somewhat serious about it,no


In [140]:
more_matches(lover_recommender_test8(invalue7, religion7, lowest_age7, highest_age7), religion7, lowest_age7, highest_age7)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


age                                       35
sex                                        f
orientation                         bisexual
body_type                              curvy
diet                         mostly anything
drinks                                 often
drugs                                  never
offspring      has kids, and might want more
pets                                has dogs
religion                        christianity
smokes                                   yes
Name: 25664, dtype: object


Do you like them? (yes/no) yes


age                                         32
sex                                          f
orientation                                gay
body_type                              average
diet                                  anything
drinks                                socially
drugs                                    never
offspring                            has a kid
pets           dislikes dogs and dislikes cats
religion                          christianity
smokes                                      no
Name: 49211, dtype: object


Do you like them? (yes/no) yes


...

Do you like them? (yes/no) no


C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Do you want to see more matches? (yes/no) no


Here's everyone you matched with:


,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
25664,35,f,bisexual,curvy,mostly anything,often,never,"has kids, and might want more",has dogs,christianity,yes
49211,32,f,gay,average,anything,socially,never,has a kid,dislikes dogs and dislikes cats,christianity,no
24990,32,f,bisexual,average,anything,rarely,never,has kids,dislikes dogs and dislikes cats,christianity and somewhat serious about it,sometimes
47403,44,f,gay,average,anything,rarely,never,doesn't have kids,likes dogs and has cats,christianity and somewhat serious about it,sometimes
34449,41,f,bisexual,curvy,anything,rarely,sometimes,doesn't have kids,likes dogs and likes cats,christianity and very serious about it,trying to quit


In [205]:
# gay male

invalue9 = ['25', 'm', 'gay', 'thin', 'vegan', 'no', 'no', "doesn't have kid(s), but wants kid(s)", 'likes dogs and cats', "agnosticism", 'sometimes']
highest_age9 = 32
lowest_age9 = 28
religion9 = "agnosticism"

lover_recommender_test8(invalue9, religion9, lowest_age9, highest_age9)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


"Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment."

In [445]:
# IT IS THIS ONE

invalue_df = pd.DataFrame(columns = cupid.columns)

matches = lover_recommender_test8(invalue, religion, lowest_age, highest_age)

Flag = 'yes'
while Flag == 'yes':
    for i in range(len(matches)):
        person = matches.iloc[i]
        if pd.DataFrame(person).T.index[0] not in invalue_df.index:
            print(person)
            answer = str(input('Do you like them?'))
            if answer == 'yes':
                invalue_df = invalue_df.append(person)
            else:
                continue

    # convert the people they like into a df
    #invalue_df = pd.DataFrame(likes).rename(columns = {i:j for i,j in zip(np.arange(11), cupid_df.columns)})

    # straight female
    if invalue_df['sex'].unique()[0] == 'f' and invalue_df['orientation'].unique()[0] == 'straight':
        straight_female = cupid_df[(cupid_df['sex'] == 'f') & (cupid_df['orientation'] == 'straight') & (cupid_df['religion'] == religion) & \
            (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)].head(10000)
        
        matches = invalue_to_similarity(invalue_df, straight_female)

    # straight male
    elif invalue_df['sex'].unique()[0] == 'm' and invalue_df['orientation'].unique()[0] == 'straight':
        straight_male = cupid_df[(cupid_df['sex'] == 'm') & (cupid_df['orientation'] == 'straight') & (cupid_df['religion'] == religion) & \
            (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)].head(10000)
        
        matches = invalue_to_similarity(invalue_df, straight_male)

    # gay female
    elif invalue_df['sex'].unique()[0] == 'f' and invalue_df['orientation'].unique()[0] == 'gay':
        gay_female = cupid_df[(cupid_df['sex'] == 'f') & (cupid_df['orientation'] == 'gay') & (cupid_df['religion'] == religion) & \
            (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
        
        matches = invalue_to_similarity(invalue_df, gay_female)

    # gay male
    elif invalue_df['sex'].unique()[0] == 'm' and invalue_df['orientation'].unique()[0] == 'gay':
        gay_male = cupid_df[(cupid_df['sex'] == 'm') & (cupid_df['orientation'] == 'gay') & (cupid_df['religion'] == religion) & \
            (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
        
        matches = invalue_to_similarity(invalue_df, gay_male)

    # bi individual
    elif (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'f') or \
         (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'm'):
        bi = cupid_df[((cupid_df['orientation'] == 'bisexual') | (cupid_df['orientation'] == 'gay')) & (cupid_df['religion'] == religion) & \
                  (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
        
        matches = invalue_to_similarity(invalue_df, bi)

    Flag = str(input('Do you want to see more matches?'))
    
print("Here's everyone you matched with:")
invalue_df

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


age                           26
sex                            f
orientation             straight
body_type               athletic
diet                    anything
drinks                     often
drugs                      never
offspring      doesn't have kids
pets                    has cats
religion            christianity
smokes                        no
Name: 1857, dtype: object


Do you like them? no


age                                                 27
sex                                                  f
orientation                                   straight
body_type                                      average
diet                                   mostly anything
drinks                                          rarely
drugs                                            never
offspring      doesn't have kids, and doesn't want any
pets                                          has cats
religion                                  christianity
smokes                                              no
Name: 25124, dtype: object


Do you like them? yes


...

Do you like them? yes


C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Do you want to see more matches? no


Here's everyone you matched with:


,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
25124,27,f,straight,average,mostly anything,rarely,never,"doesn't have kids, and doesn't want any",has cats,christianity,no
41612,34,f,straight,rather not say,anything,often,never,doesn't have kids,likes cats,christianity,no
5305,34,f,straight,rather not say,anything,not at all,never,doesn't have kids,dislikes dogs and dislikes cats,christianity and somewhat serious about it,no
41594,34,f,straight,rather not say,anything,socially,never,doesn't have kids,likes dogs,christianity,no
41624,25,f,straight,rather not say,anything,not at all,never,doesn't have kids,has cats,christianity and very serious about it,no
41521,25,f,straight,rather not say,anything,socially,never,doesn't have kids,dislikes dogs and dislikes cats,christianity,no
44769,25,f,straight,rather not say,anything,socially,never,doesn't have kids,dislikes dogs and dislikes cats,christianity,no
46967,25,f,straight,rather not say,anything,socially,never,doesn't have kids,dislikes dogs and dislikes cats,christianity,no
49804,25,f,straight,rather not say,anything,socially,never,doesn't have kids,has dogs,christianity and somewhat serious about it,no
